# PODMAN IN ACTION: Secure, rootless containers for Kubernetes, microservices, and more

by Daniel Walsh

- Publisher : Manning (February 7, 2023)
- Language : English
- Paperback : 312 pages
- ISBN-10 : `1633439682`
- ISBN-13 : `978-1633439689`

# 1. <b>Podman: A next-generation container engine</b>

## 1.1 About all these terms

Before you go further, I think it is important to define the terminology that will be used throughout this book. In the container world, terms like container orchestrator, container engine, and container runtime are often used interchangeably, which commonly leads to confusion. The following list is a summary of what each of these terms refers to in the context of this text:

### Container orchestrators

Software projects and products that orchestrate containers onto multiple different machines or nodes. These orchestrators communicate with **container engines** to run containers. The primary container orchestrator is **Kubernetes**, which was originally designed to talk to the Docker daemon container engine, but using Docker is becoming obsolete because Kubernetes primarily uses CRI-O or containerd as its container engine. **CRI-O** and **containerd** are purpose built for running orchestrated Kubernetes containers (CRI-O is covered in appendix A). Docker Swarm and Apache Mesos are other examples of container orchestrators.

#### CRI-O

_ChatGPT:_  
**CRI-O** is an open-source, lightweight container runtime for Kubernetes. It is designed to provide a stable, secure, and scalable environment for running containers in a Kubernetes cluster. CRI-O is optimized for use with Kubernetes and supports the **container runtime interface (CRI)** standard, allowing it to seamlessly integrate with the Kubernetes ecosystem. It is an alternative to other container runtimes like Docker and is specifically built to meet the requirements of Kubernetes container orchestration. CRI-O focuses on minimalism, performance, and security, making it a popular choice for running containers in production environments.

#### Container Runtime Interface

_ChatGPT:_  
**Container Runtime Interface (CRI)** is an interface between 
- container runtimes and 
- container management systems (e.g. Kubernetes). 

It allows container runtimes to be pluggable and allows different container runtimes to be used with the same container management system. This flexibility enables organizations to choose the best container runtime for their needs and easily switch between runtimes as necessary. CRI defines a set of APIs that container runtimes must implement in order to be compatible with container orchestration systems like Kubernetes.

#### Containerd

_ChatGPT:_  
**Containerd** is an open-source container runtime that provides a runtime environment for running containerized applications. It is designed to be lightweight, portable, and platform agnostic, making it suitable for use in various environments, including cloud, on-premises, and edge computing.

Containerd is used as the container runtime within Docker, as well as by other container orchestration platforms such as Kubernetes. It provides core container runtime functionalities, such as image distribution, container execution, and storage management. Containerd is designed with a modular architecture, allowing it to be integrated into different container management systems and to support various container formats.

In summary, Containerd is a fundamental component in the container ecosystem, providing essential container runtime functionalities while enabling interoperability and versatility across different container platforms.

#### Containerd vs CRI-O

- Architecture:
    - **Containerd:** Containerd is a core container runtime that provides a runtime environment for running containerized applications. It is designed to be lightweight and platform-agnostic, focusing on managing the container lifecycle, including 
        - image distribution, 
        - container execution, and 
        - storage management.  
        Containerd can be used as a building block for higher-level container orchestration systems.
    - **CRI-O:** CRI-O, on the other hand, is specifically designed for Kubernetes container runtimes. It implements the Container Runtime Interface (CRI) and focuses on creating an optimized runtime for Kubernetes pods and containers. CRI-O is specifically tailored for integrating with Kubernetes and adheres to the Kubernetes container runtime standards.<br><br>
- Integration with Kubernetes:
    - **Containerd:** Containerd can be integrated with Kubernetes as a container runtime, providing the core functionality required for running containers within a Kubernetes cluster.
    - **CRI-O:** CRI-O is designed as a lightweight and optimized runtime for Kubernetes. It directly implements the CRI, allowing Kubernetes to use CRI-O as a container runtime without the need for additional layers.<br><br>
- Feature Set:
    - **Containerd:** As a fundamental container runtime, Containerd provides core container lifecycle management functionalities, including image distribution, container execution, and storage management. It focuses on being modular and platform-agnostic, suitable for use in various container management systems.
    - **CRI-O:** CRI-O is specifically tailored to address the requirements of Kubernetes. It is designed to efficiently handle Kubernetes pod and container management, adhering closely to the Kubernetes container runtime standards while providing essential container runtime features.

In summary, Containerd is a general-purpose, lightweight container runtime suitable for various container management systems, while CRI-O is specifically optimized for Kubernetes, providing a streamlined and Kubernetes-focused runtime environment. Both container runtimes have unique strengths based on their intended use cases and integrations.

### Container engines

Primarily used for configuring containerized applications to run _on a single local node_. They can be launched directly by users, administrators, and developers. They can also be launched out of systemd unit files at boot as well as launched by **container orchestrators** like Kubernetes. As previously mentioned, `CRI-O` and `containerd` are container engines (_runtimes_ - VR) used by Kubernetes to manage containers locally. They really are not intended to be used directly by users. `Docker` and `Podman` are the primary **container engines** used by users to develop, manage, and run containerized applications on a single machine. Podman is seldom used to launch containers for Kubernetes; therefore, Kubernetes is not generally covered in this book. `Buildah` is another container engine, although it is only used for building container images.

### Open Container Initiative (OCI) container runtimes

Configure different parts of the Linux kernel and then, finally, launch the containerized application. The two most commonly used container runtimes are `runc` and `crun`. `Kata` and `gVisor` are other examples of container runtimes. See appendix B to understand the differences between the OCI container runtimes.

_ChatGPT:_  
`runc` is not a full-fledged container runtime, but a lightweight command-line tool for spawning and running containers. It is a low-level tool used by higher-level container runtimes (like CRI-O and containerd) to create and run containers based on OCI (Open Container Initiative) specifications.

### Podman

**Podman** is short for _Pod Manager_. A **pod**, a concept popularized by the Kubernetes project, is one or more containers sharing the same namespaces and cgroups (resource constraints). Pods are covered in greater depth in **chapter 4**. Podman runs individual containers as well as pods. The Podman logo is a group of Selkies, the Irish concept of a mermaid. Groups of Selkies are called pods.

![](../data/images/podman-logo-2009470014.png)

![](../data/images/fgfff54fdgdg.jpg)

![](../data/images/048c64b76bdb2b40157dc81f6d246e96.png)

## 1.2 A brief overview of containers

### Containers

**Containers** are groups of _processes_ running on a Linux system that are isolated from each other. 

Containers make sure one group of processes does not interfere with other processes on the system. Rogue processes can’t dominate system resources, which might prevent other processes from performing their task. Hostile containers are also prevented from attacking other containers, stealing data, or causing denial of service attacks. 

> A final goal of containers is allowing applications to be installed with their own versions of shared libraries that do not conflict with applications requiring different versions of the same libraries. 

Instead they allow applications to live in a virtualized environment, giving the impression that they own the entire system.

Containers are isolated via the following:

#### Resource constraints (cgroups)

The [cgroup man page](https://man7.org/linux/man-pages/man7/cgroups.7.html) defines cgroups as the following: 

_“Control groups, usually referred to as cgroups, are a Linux kernel feature which allow processes to be organized into hierarchical groups whose usage of various types of resources can then be limited and monitored.”_

Examples of resources controlled by cgroups include the following:

- The amount of **memory** a group of processes can use,
- The amount of **CPU** processes can use,
- The amount of **network** resources a process can use.

The basic idea of cgroups is 

> preventing one group of processes from dominating certain system resources in such a way that another group of processes can’t make progress on the system.

In [12]:
man cgroups

cgroups(7)             Miscellaneous Information Manual             cgroups(7)

NAME
       cgroups - Linux control groups

DESCRIPTION
       Control groups, usually referred to as cgroups, are a Linux kernel fea‐
       ture which allow processes to be  organized  into  hierarchical  groups
       whose usage of various types of resources can then be limited and moni‐
       tored.  The kernel's cgroup interface is  provided  through  a  pseudo-
       filesystem called cgroupfs.  Grouping is implemented in the core cgroup
       kernel code, while resource tracking and limits are  implemented  in  a
       set of per-resource-type subsystems (memory, CPU, and so on).

   Terminology
       A cgroup is a collection of processes that are bound to a set of limits
       or parameters defined via the cgroup filesystem.

       A subsystem is a kernel component that modifies  the  behavior  of  the
       processes  in a cgroup.  Various subsystems have been implemented, mak‐
       ing 

#### Security constraints

Containers are isolated from each other using many security tools available in the kernel. The goal is 

> blocking privilege escalation and preventing a rogue group of processes from committing hostile acts against the system, including the following examples:

- Dropped Linux capabilities limit the power of root.
- SELinux controls access to the filesystem.
- There is read-only access to kernel filesystems.
- Seccomp limits the system calls available in the kernel.
- A user namespace to map one group of UIDs in the host to another allows access to limited root environments.

### Namespaces - Virtualization technologies

The Linux kernel employs a concept called **namespaces**, which creates virtualized environments, where one set of processes sees one set of resources, while another set of processes sees a different set of resources. These **virtualized environments** eliminate processes’ views into the rest of the system, giving them the feel of a virtual machine (VM) without the overhead. Examples of namespaces include the following:

- **Network namespace** — Eliminates the access to the host network but gives access to virtual network devices,
- **Mount namespace** — Eliminates the view of all the filesystem, except the containers filesystem,
- **PID namespace** — Eliminates the view of other processes on the system; container processes only see the processes within the container.

These container technologies have existed in the Linux kernel for many years. Security tools for isolating processes started in Unix back in the 1970s, and SELinux started in 2001. Namespaces were introduced around 2004, and cgroups were introduced around 2006.

### Container image format

A **container image** consists of three components:

- A **directory tree** containing all the software required to run your application,
- A JSON file that describes the contents of the **rootfs**,
- Another JSON file called a **manifest list** that links multiple images together to support different architectures.

The directory tree is called a **rootfs** (root filesystem). The software is laid out like it was the root (`/`) of a Linux system.

The executable to be run within the `rootfs`, the working directory, the environment variables to be used, the maintainer of the executable, and other labels to help identify the content of the image are defined in the first JSON file. You can see this JSON file using the `podman inspect` command:

```sh
$ podman inspect docker://registry.access.redhat.com/ubi8
```
```json
{
...
  "created": "2022-01-27T16:00:30.397689Z",      ❶
  "architecture": "amd64",                       ❷
  "os": "linux",                                 ❸
  "config": {
         "Env": [                                ❹
            "PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin",
            "container=oci"
         ],
         "Cmd": [                                ❺
                   "/bin/bash"
         ],
         "Labels": {                             ❻
                     "architecture": "x86_64",
                     "build-date": "2022-01-27T15:59:52.415605",
       ...
}
```

1. Date the image was created
1. Architecture for this image
1. Operating system for this image
1. Environment variables that the developer of the image wants to be set within the container
1. Default command to be executed when the container starts
1. Labels to help describe the contents of the image. These fields can be free-form and do not affect the way images are run but can be used to search for and describe the image.

The second JSON file, the **manifest list**, allows users on an arm64 machine to pull an image with the same name as they would if they were on an arm64 machine. Podman pulls the image based on the default architecture of the machine, using this manifest list:

```json
{
    "manifests": [
      {
              "digest": "sha256:cbc1e8cea
➥ 8c78cfa1490c4f01b2be59d43ddbb
➥ ad6987d938def1960f64bcd02c",                                                    ❶
              "mediaType": "application/vnd.docker.distribution.manifest.v2+json",❷
              "platform": {
              "architecture": "amd64",                                            ❸
              "os": "linux"                                                       ❹
              },
              "size": 737
      },
      {
              "digest":                                                           ❺
➥ "sha256:f52d79a9d0a3c23e6ac4c3c8f2ed8d6337ea47f4e2dfd46201756160ca193308",
              "mediaType": "application/vnd.docker.distribution.manifest.v2+json",
              "platform": {
              "architecture": "arm64", 
              "os": "linux"
              },
              "size": 737
      },
...
}
```

1. Digest (hash sum) of the exact image pulled when the architecture and OS match
1. mediaType describes the type of the image, OCI, Docker, and so on.
1. The architecture of this image digest: amd64
1. The OS of this image digest: Linux
1. This stanza (строка файла конфигурации) points to a different image for a different architecture: arm64.

### <b>Skopeo</b>

**Skopeo** (Greek for "remote viewing") is a tool that uses the same underlying libraries as Podman and is available at [github.com/containers/skopeo](github.com/containers/skopeo) (see appendix A). Skopeo provides lower-level output examining the structures of a container image. 

Skopeo works with Podman and Buildah to manage OCI containers. Put simply, 
- **Podman** runs containers, 
- **Buildah** builds containers, and 
- **Skopeo** transports containers –– among other things. 

Think of these tools as a swiss army knife for your container environment. Skopeo is a deft and versatile blade at your disposal. ([source](https://www.redhat.com/en/topics/containers/what-is-skopeo))

In the following example, use the `skopeo` command with the `--raw` option to examine the [registry.access.redhat.com/ubi8](registry.access.redhat.com/ubi8) image manifest specification without the need to pull the image:

In [11]:
# sudo apt update && sudo apt install skopeo -y

skopeo inspect --raw docker://registry.access.redhat.com/ubi8 | jq

{
  "manifests": [
    {
      "digest": "sha256:4b5ad3e7af158ded8070d4e0f1d75a3f197d065efeadddf33455f486f30e7832",
      "mediaType": "application/vnd.docker.distribution.manifest.v2+json",
      "platform": {
        "architecture": "amd64",
        "os": "linux"
      },
      "size": 429
    },
    {
      "digest": "sha256:5c33b45ca04495caaa41eaae45e471748480d44fd39aa09350db45bbdeb40f55",
      "mediaType": "application/vnd.docker.distribution.manifest.v2+json",
      "platform": {
        "architecture": "arm64",
        "os": "linux"
      },
      "size": 429
    },
    {
      "digest": "sha256:47df466911a0351a1e9744fb065bab435f7f5aec0050ad0e764fbed45b4668fd",
      "mediaType": "application/vnd.docker.distribution.manifest.v2+json",
      "platform": {
        "architecture": "ppc64le",
        "os": "linux"
      },
      "size": 429
    },
    {
      "digest": "sha256:90a0be2052eaf5adcaff2f9ccbe2e0b4eeb4421454c7a1eb19d3c9e3d7cfc0b8",
      "mediaType": "application/vnd.doc

#### Postgres example (VR)

In [9]:
podman search postgres

NAME                                                 DESCRIPTION
docker.io/library/postgres                           The PostgreSQL object-relational database sy...
docker.io/bitnami/postgresql                         Bitnami PostgreSQL Docker Image
docker.io/cimg/postgres                              
docker.io/bitnami/postgres-exporter                  
docker.io/bitnami/postgresql-repmgr                  
docker.io/ubuntu/postgres                            PostgreSQL is an open source object-relation...
docker.io/rapidfort/postgresql                       RapidFort optimized, hardened image for Post...
docker.io/rapidfort/postgresql-official              RapidFort optimized, hardened image for Post...
docker.io/bitnamicharts/postgresql                   
docker.io/rapidfort/postgresql12-ib                  RapidFort optimized, hardened image for Post...
docker.io/cockroachdb/postgres-test                  An environment to run the CockroachDB accept...
docker.io/elestio/postgres  

In [9]:
podman images

REPOSITORY  TAG         IMAGE ID    CREATED     SIZE


Get the manifest file:

In [10]:
skopeo inspect --raw docker://docker.io/library/postgres | jq

{
  "manifests": [
    {
      "annotations": {
        "org.opencontainers.image.revision": "d416768b1a7f03919b9cf0fef6adc9dcad937888",
        "org.opencontainers.image.source": "https://github.com/docker-library/postgres.git#d416768b1a7f03919b9cf0fef6adc9dcad937888:16/bookworm",
        "org.opencontainers.image.url": "https://hub.docker.com/_/postgres",
        "org.opencontainers.image.version": "16.1"
      },
      "digest": "sha256:60c91e0203ae5ccca0a251953742752cd16a129db181bc15559cca71420b188c",
      "mediaType": "application/vnd.oci.image.manifest.v1+json",
      "platform": {
        "architecture": "amd64",
        "os": "linux"
      },
      "size": 3324
    },
    {
      "annotations": {
        "vnd.docker.reference.digest": "sha256:60c91e0203ae5ccca0a251953742752cd16a129db181bc15559cca71420b188c",
        "vnd.docker.reference.type": "attestation-manifest"
      },
      "digest": "sha256:d75fb3e8a4753c68a7ab34ebe8fef6b7a9c1daa76f202a9fc6fef45c9b844b82",
      "medi

Get the overall information of the repo:

In [6]:
skopeo inspect docker://docker.io/library/postgres | jq

{
  "Name": "docker.io/library/postgres",
  "Digest": "sha256:49c276fa02e3d61bd9b8db81dfb4784fe814f50f778dce5980a03817438293e3",
  "RepoTags": [
    "10",
    "10-alpine",
    "10-alpine3.13",
    "10-alpine3.14",
    "10-alpine3.15",
    "10-alpine3.16",
    "10-beta1",
    "10-beta1-alpine",
    "10-beta2",
    "10-beta2-alpine",
    "10-beta3",
    "10-beta3-alpine",
    "10-beta4",
    "10-beta4-alpine",
    "10-bullseye",
    "10-buster",
    "10-rc1",
    "10-rc1-alpine",
    "10-stretch",
    "10.0",
    "10.0-alpine",
    "10.1",
    "10.1-alpine",
    "10.10",
    "10.10-alpine",
    "10.11",
    "10.11-alpine",
    "10.12",
    "10.12-alpine",
    "10.13",
    "10.13-alpine",
    "10.14",
    "10.14-alpine",
    "10.15",
    "10.15-alpine",
    "10.16",
    "10.16-alpine",
    "10.17",
    "10.17-alpine",
    "10.17-alpine3.13",
    "10.17-alpine3.14",
    "10.17-buster",
    "10.17-stretch",
    "10.18",
    "10.18-alpine",
    "10.18-alpine3.14",
    "10.18-bullseye",
    "

Skip annoying tags:

In [5]:
skopeo inspect --no-tags docker://docker.io/library/postgres | jq

{
  "Name": "docker.io/library/postgres",
  "Digest": "sha256:49c276fa02e3d61bd9b8db81dfb4784fe814f50f778dce5980a03817438293e3",
  "RepoTags": [],
  "Created": "2024-01-04T21:52:40Z",
  "DockerVersion": "",
  "Labels": null,
  "Architecture": "amd64",
  "Os": "linux",
  "Layers": [
    "sha256:2f44b7a888fa005d07c031d3cfad2a1c0344207def2ab9dbb97712425ff812c1",
    "sha256:6d49150dabe2486c7b35dbe0ea864b690e079fa1b48ef77d1e4533d96b4051e7",
    "sha256:18d6a86d0fbff788e398befc7b6139bed448853e3a113ea6b2c6c97783f4d8a1",
    "sha256:4c9385c30bce0478ebb3d7b8cd3ae66e9b211e980669bd0d67f3bb38b8bfaa40",
    "sha256:550091272acc768f00ff9a7ae977f9c558f5d3364cd5f87fabcc452d6f677b0e",
    "sha256:2720859ac49e9505ac9b95a3937053a3e0c2bb623b4bb6f49468c20ef7db8bc4",
    "sha256:b8091cf535458ca73ced6bb49c25ec769ae88608216e721e6bd1767d36138e3f",
    "sha256:f3ca5fbd89cdb8212ec43875a45b14a9f1cfda38272896347251e76e0cc383fe",
    "sha256:22fbbce47a56e5ceed3d8ec4a588a1adaed23a83c230645206ec26d961f876cd",
    "s

Get only layers:

In [18]:
skopeo inspect docker://docker.io/library/postgres | jq .Layers

[
  "sha256:2f44b7a888fa005d07c031d3cfad2a1c0344207def2ab9dbb97712425ff812c1",
  "sha256:6d49150dabe2486c7b35dbe0ea864b690e079fa1b48ef77d1e4533d96b4051e7",
  "sha256:18d6a86d0fbff788e398befc7b6139bed448853e3a113ea6b2c6c97783f4d8a1",
  "sha256:4c9385c30bce0478ebb3d7b8cd3ae66e9b211e980669bd0d67f3bb38b8bfaa40",
  "sha256:550091272acc768f00ff9a7ae977f9c558f5d3364cd5f87fabcc452d6f677b0e",
  "sha256:2720859ac49e9505ac9b95a3937053a3e0c2bb623b4bb6f49468c20ef7db8bc4",
  "sha256:b8091cf535458ca73ced6bb49c25ec769ae88608216e721e6bd1767d36138e3f",
  "sha256:f3ca5fbd89cdb8212ec43875a45b14a9f1cfda38272896347251e76e0cc383fe",
  "sha256:22fbbce47a56e5ceed3d8ec4a588a1adaed23a83c230645206ec26d961f876cd",
  "sha256:b3b5e3b65b9594578082fe4f45b479c2de5782344f17d74b7e34a3184000bf28",
  "sha256:917e5b76e085bfd128b12de93138dc43366eefc6efe52bedea5d351a0612b50f",
  "sha256:7f21ce9572c6e1750cc335372d0d1c6a893da69e3064e7efc961237c5ba63538",
  "sha256:4ea3941c8572e8e5721645ca86d9b9ddc868a5276c899e036d6da84fba9890ec

In [9]:
skopeo layers docker://docker.io/library/postgres | jq

DEPRECATED: skopeo layers is deprecated in favor of skopeo copy



Count the number of tags:

In [8]:
skopeo list-tags docker://docker.io/library/postgres | wc -l

828


#### Mans of Skopeo

In [13]:
man skopeo

SKOPEO(1)(Skopeo)                                            SKOPEO(1)(Skopeo)

Jhon Honce August 2016

NAME
       skopeo  --  Command line utility used to interact with local and remote
       container images and container image registries

SYNOPSIS
       skopeo [global options] command [command options]

DESCRIPTION
       skopeo is a command line utility providing various operations with con‐
       tainer images and container image registries.

       skopeo  can  copy  container  images  between  various containers image
       stores, converting them as necessary.  For example you can  use  skopeo
       to copy container images from one container registry to another.

       skopeo  can convert a Docker schema 2 or schema 1 container image to an
       OCI image.

       skopeo can inspect a repository on a container registry  without  need‐
       lessly  pulling  the  image.  Pulling an image from a repository, espe‐
       cially a remote repository, is an expensive networ

In [4]:
man skopeo-inspect

skopeo-inspect(1)()                                        skopeo-inspect(1)()

NAME
       skopeo-inspect  -  Return  low-level  information about image-name in a
       registry.

SYNOPSIS
       skopeo inspect [options] image-name

DESCRIPTION
       Return low-level information  about  image-name  in  a  registry.   See
       skopeo(1) for the format of image-name.

       The  default  output  includes  data  from  various sources: user input
       (Name), the remote repository, if any (RepoTags), the  top-level  mani‐
       fest  (Digest),  and  a  per-architecture/OS image matching the current
       run-time environment (most other values).  To see values for a  differ‐
       ent  architecture/OS,  use  the --override-os / --override-arch options
       documented in skopeo(1).

OPTIONS
       --authfile path

       Path of the authentication file. Default is ${XDG_RUNTIME_DIR}/contain‐
       ers/auth.json,  which  is set using skopeo login.  If the authorization
       s

In [10]:
man skopeo copy

skopeo-copy(1)()                                              skopeo-copy(1)()

NAME
       skopeo-copy  -  Copy an image (manifest, filesystem layers, signatures)
       from one location to another.

SYNOPSIS
       skopeo copy [options] source-image destination-image

DESCRIPTION
       Copy an image (manifest, filesystem layers, signatures) from one  loca‐
       tion to another.

       Uses  the  system's trust policy to validate images, rejects images not
       trusted by the policy.

       source-image use the "image name" format described above

       destination-image use the "image name" format described above

       source-image and destination-image are interpreted completely  indepen‐
       dently;  e.g.  the  destination name does not automatically inherit any
       parts of the source name.

OPTIONS
       --additional-tag=strings

       Additional tags (supports docker-archive).

       --all, -a

       If source-image refers to a list of images, instead of cop

### Container registries

Images use the Linux `tar` utility to pack the `rootfs` and the JSON files together. These images are then stored on web servers called **container registries** (e.g., `docker.io`, `quay.io`, and `Artifactory`). **Container engines** like Podman can copy these images to a host and unpack them onto the filesystem. Then the engine merges 
- the image’s JSON file, 
- the engine’s built-in defaults, and 
- the user’s input 

to create a new container OCI runtime specification JSON file. The JSON file describes how to run the containerized application.

### Container runtime

In the last step, the container engine launches a small program called a **container runtime** (e.g., `runc`, `crun`, `kata`, or `givisord`). The container runtime reads the 
- container’s JSON and instruments, 
- kernel cgroups, 
- security constraints, and 
- namespaces 

before finally launching the primary process of the container.

### Container standards

The OCI standards body defined the standard formats for storing and defining container images. They also defined the standard for container engines running containers. The OCI created the OCI Image Format, which standardizes the format of the container images and the images’ JSON file. They also created the OCI Runtime Specification, which standardized the container’s JSON file to be used by OCI runtimes. The OCI standards allow other container engines, like Podman, to follow the standards and be able to work with all the images stored at container registries and to run them in the exact same way as all other container engines, including Docker (see figure 1.7).

### Pictures

<center>
    <img src="../data/images/Screenshot_20240114_010033.png" alt="Figure 1.4 Physical machine running three applications in three VMs" style="width: 450px">
    <p style="text-align: center"><i>Figure 1.4 Physical machine running three applications in three VMs</i></p>

<center>
    <img src="../data/images/Screenshot_20240114_010239.png" alt="Figure 1.5 Physical machine running three applications in three containerized applications" style="width: 450px">
    <p style="text-align: center"><i>Figure 1.5 Physical machine running three applications in three containerized applications</i></p>

<center>
    <img src="../data/images/Screenshot_20240114_010402.png" alt="Figure 1.6 Traditional LAMP stack (Linux, Apache, MariaDB, and PHP/PERL application) running on a server" style="width: 450px">
    <p style="text-align: center"><i>Figure 1.6 Traditional LAMP stack (Linux, Apache, MariaDB, and PHP/PERL application) running on a server</i></p>

<center>
    <img src="../data/images/Screenshot_20240114_005714.png" alt="Figure 1.7 LAMP stack packaged individually into microservice containers. As containers communicate via the network, they can be easily moved to other VMs, making reuse much easier." style="width: 450px">
    <p style="text-align: center"><i>Figure 1.7 LAMP stack packaged individually into microservice containers. As containers communicate via the network, they can be easily moved to other VMs, making reuse much easier.</i></p>

## 1.3 `Fork`/`exec` model

Docker is built as a REST API server. Fundamentally Docker is a client-server architecture including multiple daemons. When a user executes the Docker client, they execute a command-line tool that connects to the Docker daemon. The Docker daemon then pulls images to its storage and then connects to the containerd daemon, which finally executes an OCI runtime that creates the container. The Docker daemon, then, is a communication platform that communicates reads and writes of `stdin`, `stdout`, and `stderr` from the initial process (PID1) created in the container. The daemon relays all of the output back to the Docker client. Users imagine the container’s processes are just children of the current session, but there is a lot of communication going on behind the scenes. Figure 1.8 shows the Docker client-server architecture (also see [Container](../containers_nomenclature.ipynb#Container) from "Containers Nomenclature").

The bottom line is the Docker client communicates with the Docker daemon, which then communicates with the containerd daemon, which finally launches an OCI runtime like runc to launch PID1 of the container. There is a lot of complexity involved in running containers in this way. Over the years, failures in any of the Daemons have led to all containers shutting down, and it is often difficult to diagnose what happened. The core Podman engineering team comes from an operating system background grounded in the Unix philosophy.

Unix and C were designed with the **fork/exec model** of computing. Basically, when you execute a new program, a parent program like the Bash shell _forks_ a new process and then executes the new program as a child of the old program. The Podman engineering team thought they could make containers simpler by building a tool that pulls container images from a container registry, configures container storage, and then launches an OCI runtime, which starts the container as a child of your container engine.

In the Unix operating system, processes can share content via the filesystem and inter-process communication (IPC) mechanisms. These features of the operating system enable multiple container engines to share storage without requiring a daemon to be running to control access and share content. The engines do not need to communicate together aside from using locking mechanisms provided by the operating system’s filesystems. Future chapters examine the advantages and disadvantages of this mechanism. Figure 1.9 shows the Podman architecture and communication flow.

<center>
    <div style="display: flex; flex-direction: row">
        <div>
            <img src="../data/images/Screenshot_20240114_015956.png" alt="Figure 1.8 Docker client-server architecture. The container is a direct descendant of containerd, not the Docker client. The kernel sees no relationship between the client program and the container." style="width: 600px; padding: 5px;">
            <p style="text-align: center; padding-top: 0px"><i>Figure 1.8 Docker client-server architecture. The container is a direct descendant of containerd, not the Docker client. The kernel sees no relationship between the client program and the container.<i></p>
        </div>
        <div>
            <img src="../data/images/Screenshot_20240114_020023.png" alt="Figure 1.9 Podman fork/exec architecture. The user launches Podman, which executes the OCI runtime, which then launches the container. The container is a direct descendant of Podman." style="width: 250px; padding: 5px;">
            <p style="text-align: center; padding-top: 0px"><i>Figure 1.9 Podman fork/exec architecture. The user launches Podman, which executes the OCI runtime, which then launches the container. The container is a direct descendant of Podman.</i></p>
        </div>
    </div>

Imagine you have a web service that you want to run at boot time. The web service is packaged in a container, so you need a container engine. In the Docker case, you need to set it up to be running on your machine with each of the daemons running and accepting connections. Next, launch the Docker client to start the web service. Now you have your containerized application running as well as all of the Docker daemons. In the Podman case, use the Podman command to launch your container, and Podman will go away. Your container will continue to run without the overhead of running the multiple daemons. Less overhead is incredibly popular on low-end machines like IOT devices and edge servers.

## 1.3.7 Integration with systemd

Systemd is the fundamental init system in the operating systems. The init process on a Linux system is the first process that is started by the kernel on boot. Therefore, the init system is the ancestor of all processes and can monitor them all. Podman wants to fully integrate the running of containers with the init system. Users want to use systemd to start and stop containers at boot time. Containers should do the following:

- Support systemd within a container
- Support socket activation
- Support systemd notifications that a containerized application is fully activated
- Allow systemd to fully manage the cgroups and lifespan of a containerized application

Basically, containers work as services in systemd unit files. Many developers want to run systemd within a container to run multiple system-defined services within a container.

However, the upstream Docker community disagrees with this and has denied all pull requests that attempt to integrate systemd into Docker. They believe Docker should manage the life cycle of the container, and they do not want to accommodate users who want to run systemd in a container.

The upstream Docker community believes the Docker daemon, as opposed to systemd, should be the controller of processes, it should manage the life cycle of containers, and it should start and stop them at boot time. The problem is there are many more features in systemd than in Docker, including startup ordering, socket activation, service ready notifications, and so on.

When Podman was designed, the developers wanted to make sure it fully integrated with systemd. When you run systemd inside a container, Podman sets up the container the way systemd expects and allows it to simply run as PID1 of the container with limited privileges. Podman allows you to run services within the container the same way they run on a system or in a VM: via systemd unit files. Podman supports socket activation, service notifications, and many other systemd unit file features. Podman makes it simple to generate systemd unit files with best practices for running containers within a systemd service. For more information, see chapter 7 on systemd integration.

The Containers project (https://github.com/containers) where Podman, container libraries, and other container management tools reside, wants to embrace all features of the operating system and fully integrate it. Chapter 7 explains Podman integration with systemd.

## 1.3.8 Pods

One advantage of Podman is described in its name. As mentioned earlier, Podman is actually short for **Pod Manager**. As the official Kubernetes documentation puts it, 

> “A **pod** (as in a pod of seals, hence the logo, or pea pod) is a group of one or more containers, with shared storage/network resources, and a specification for how to run the containers.” 

Podman works with 
- either a single container at a time, like Docker, 
- or it can manage groups of containers together in a pod. 

One of the design goals of containers is 

> to separate services into single containers: **microservices**. Then you combine containers together to build larger services. 

Pods allow you to group multiple services together to form a larger service managed as a single entity. One of the goals of Podman is allowing you to experiment with pods. Figure 1.12 shows two pods running on a system, each pod containing three containers.

Podman has the `podman generate kube` command, which allows you to generate Kubernetes YAML files from running containers and pods, as you can see in chapter 7. Similarly, it has the `podman play kube` command, which allows you to play Kubernetes YAML files and generate pods and containers on your host. 

> I suggest using Podman for running pods and containers on a single host and using Kubernetes to take your pods and containers and run them on multiple machines and all through your infrastructure. 

Other projects, like kind (https://kind.sigs.k8s.io/docs/user/rootless), are experimenting with running pods with Podman under the guidance of Kubernetes.

<center>
    <img src="../data/images/Screenshot_20240114_022123.png" alt="Figure 1.12 Two pods running on a host. Each pod runs two different containers along with the infra container." style="width: 650px">
    <p style="text-align: center"><i>Figure 1.12 Two pods running on a host. Each pod runs two different containers along with the infra container.</i></p>

## 1.3.11 Complete customizability

Container engines tend to have lots of built-in constants, like the namespaces they run with, whether or not SELinux is enabled, and which capabilities containers run with. With Docker, most of these values are hardcoded and cannot be changed by default. Podman, on the other hand, has a very customizable configuration.

Podman has its built-in defaults but defines three locations for its configuration files to be stored:

- `/usr/share/containers/containers.conf` — Where a distribution can define the changes the distribution likes to use
- `/etc/containers/containers.conf` — Where they can set up system overrides
- `$HOME/.config/containers/containers.conf` — Can be specified only in rootless mode

The configuration files allow you to configure Podman to run the way you want by default. You can even run with more security by default if you choose.

## 1.3.12 User-namespace support

Podman is fully integrated with the user namespace. Rootless mode relies on user namespaces, which allows for multiple UIDs to be assigned to a user. User namespaces provide isolation between users on a system, so you can have multiple rootless users running containers with multiple user IDs, all isolated from each other.

A user namespace can be used to isolate containers from each other. Podman makes it simple to launch multiple containers, each with a unique user namespace. The kernel then isolates the processes from host users as well as each other based on UID separation.

Docker only supports running containers in a single, separate, user namespace, meaning all containers run within the same user namespace. Root in one container is the same as root in another container. It does not support running each container in a different user namespace, which means containers attack each other from a user-namespace perspective. Even though Docker supports this mode, almost no one runs containers with Docker in a separate user namespace.

# 2. <b>Command line</b>

- The Podman command line
- Running an OCI application
- Comparing containers and images
- Building an OCI-based image

In [4]:
man podman

podman(1)                   General Commands Manual                  podman(1)

NAME
       podman - Simple management tool for pods, containers and images

SYNOPSIS
       podman [options] command

DESCRIPTION
       Podman  (Pod  Manager)  is  a fully featured container engine that is a
       simple daemonless tool.  Podman provides a Docker-CLI  comparable  com‐
       mand  line  that  eases the transition from other container engines and
       allows the management of pods,  containers  and  images.   Simply  put:
       alias  docker=podman.   Most  Podman  commands  can be run as a regular
       user, without requiring additional privileges.

       Podman uses Buildah(1) internally  to  create  container  images.  Both
       tools share image (not container) storage, hence each can use or manip‐
       ulate images (but not containers) created by the other.

       Default settings for flags are defined in  containers.conf.  Most  set‐
       tings  for  Remote connections 

In [5]:
podman --help

Manage pods, containers and images

Usage:
  podman [options] [command]

Available Commands:
  attach      Attach to a running container
  auto-update Auto update containers according to their auto-update policy
  build       Build an image using instructions from Containerfiles
  commit      Create new image based on the changed container
  container   Manage containers
  cp          Copy files/folders between a container and the local filesystem
  create      Create but do not start a container
  diff        Display the changes to the object's file system
  events      Show podman events
  exec        Run a process in a running container
  export      Export container's filesystem contents as a tar archive
  generate    Generate structured data based on containers, pods or volumes
  healthcheck Manage health checks on containers
  help        Help about any command
  history     Show history of a specified image
  image       Manage images
  images      List images in local storage
 

## 2.1 Working with containers

Developers, administrators, quality engineers, and general users primarily use the `podman run` command to pull down and run, test, or explore these container images. To start building out containerized applications, the first thing you need to do is start working with a **base image**. In our examples, you pull and run the [registry.access.redhat.com/ubi8/httpd-24](registry.access.redhat.com/ubi8/httpd-24) image to container storage in your home directory and start exploring inside the container.

### Exploring containers

```sh
podman run -ti --rm registry.access.redhat.com/ubi8/httpd-24 bash
```

By default the podman run command executes the containerized command in the foreground until the container exits. In this case, you end up at a Bash prompt running within the container and showing the bash-4.4$ prompt. When you exit this Bash prompt, Podman stops the container.

- You used two options: `-t` and `-i`, as `-ti`, which tells Podman to hook up to the terminal. This connects to the input, output, and error stream of the bash process within the container to your screen, which allows you to interact within the container.

- The `--rm` option tells Podman to delete the container as soon as the container exits, freeing up all of the container’s storage.

- Specify the container image, [registry.access.redhat.com/ubi8/httpd-24](registry.access.redhat.com/ubi8/httpd-24), you are working with. The `podman` command reaches out to the container registry at `registry.access.redhat.com` and begins copying down the `ubi8/httpd-24:latest` image. Podman copies multiple **layers** (aka **blobs**), as shown in the following listing, and stores them in the local container storage (**container host**). You see the progress as the image layers are pulled down. Some images are rather large and can take a long time while being pulled down. If you later run a different container on the same image, Podman skips the image-pulling step, since you already have the correct image in local container storage.

- Finally, specify the executable to be run within the container, in this case, `bash`.

```
Trying to pull registry.access.redhat.com/ubi8/httpd-24:latest...
Getting image source signatures
Copying blob 89e0ad8acaf1 done  
Copying blob c2650fe947f6 done  
Copying blob 50ccdb01751a done  
Copying config 203593be2e done  
Writing manifest to image destination
Storing signatures
```
```sh
bash-4.4$ 
```

While inside the bash shell container, `cat /etc/os-release`, and notice it is likely a different OS or a different version than the `/etc/os-release` outside the container. Explore around in the container, and notice how different it is from your host environment:

```sh
bash-4.4$ grep PRETTY_NAME /etc/os-release 
```
```
PRETTY_NAME="Red Hat Enterprise Linux 8.9 (Ootpa)"
```

```sh
bash-4.4$ ls /usr/bin | wc -l    # commands available
```
```
526
```
```sh
bash-4.4$ ps
```
```
    PID TTY          TIME CMD
      1 pts/0    00:00:00 bash
      9 pts/0    00:00:00 ps
```

You can further explore the inside of the container to gain an understanding of what is going on within a container.

When you are done, you exit the bash script, and the container shuts down. Since you ran with the `--rm` option, Podman removes all the container storage and deletes the container. The container image remains in `containers/storage`:

```sh
$ sudo ls /var/lib/containers/storage/
defaultNetworkBackend  libpod  mounts  overlay  overlay-containers  overlay-images  overlay-layers  storage.lock  tmp  userns.lock

$ sudo tree /var/lib/containers/storage/
/var/lib/containers/storage/
├── defaultNetworkBackend
├── libpod
│   └── bolt_state.db
├── mounts
├── overlay
│   └── l
├── overlay-containers
│   └── containers.lock
├── overlay-images
│   └── images.lock
├── overlay-layers
│   └── layers.lock
├── storage.lock
├── tmp
└── userns.lock

9 directories, 7 files
```

### Running the containerized application

#### `podman run`

First, remove the `-ti` and the `--rm` options, since you want the container to remain running when the podman command exits. You are not a shell running within the container interactively, since it is just running the containerized web service:

```sh
$ podman run -d -p 8080:8080 --name myapp registry.access.redhat.com/ubi8/httpd-24
37a1d2e31dbf4fa311a5ca6453f53106eaae2d8b9b9da264015cc3f8864fac22
```

- `-d` (`--detach`) option tells Podman to launch the container and then detach from it. Basically, run the container in the background. The Podman command actually exits and leaves the container running. Chapter 6 goes much deeper into what is going on behind the scenes;

- `-p` (`--publish`) option tells Podman to publish or bind the container port `8080` to the host port `8080` when the container is running. With the `-p` option, the field before the colon refers to the host port, while the field after the colon refers to the container port. In this case, you see that the ports are the same. If you specify only one port, Podman considers this port a container port and randomly picks a host port on which the container port is bound. You can use the `podman port` command to discover which ports are bound to a container;

```sh
- $ podman port myapp
8080/tcp -> 0.0.0.0:8080
```

i.e. the port `8080/tcp` inside the container is bound to all of the host networks (`0.0.0.0`) at port `8080`.

By default, containers are created within their own **network namespace**, meaning they are not bound to the host network but to their virtualized network. Suppose I execute the container without the `-p` option. In that case, the Apache server within the container binds to the network interface within the container’s network namespace, but Apache is not bound to the host network.

Only processes within the container are able to connect to port `8080` to communicate with the web server. By executing the command with the `-p` option, Podman connects the port from inside the container to the host network at the specified port. The connection allows external processes like a web browser to read from the web service.

> NOTE: If you are running containers in rootless mode, covered in **chapter 3**, Podman users are by default not permitted to bind to ports `< 1024` by the kernel. Some containers want to bind to lower ports like port `80`, which is allowed inside the container, but `-p 80:80` fails, since `80` is less than `1024`. Using `-p 8080:80` causes Podman to bind the host’s port `8080` to port `80` within the container. The [upstream Podman repo](http://mng.bz/69ry) contains troubleshooting information on problems like binding to ports less than `1024` and many others.

The `-p` option can map port numbers inside the container to different port numbers outside the container.

- `--name myapp` option. Specifying a name makes it easier to find the container, and it allows you to specify a name that can then be used for other commands (e.g., `podman stop myapp`). If you don’t specify a name, Podman automatically generates a unique container name along with a container ID. All of the Podman commands that interact with containers can use either the name or the ID.

When the `podman run` command completes, the container is running. Since this container is running in detached mode, Podman prints out the container ID and exits, but the container remains running.

Now that the container is running, you can launch a web browser to communicate with the web server inside of the container at localhost port `8080` (see figure 2.1):

![Figure 2.1 Web browser window connecting to the ubi8/httpd-24 container running in Podman](../data/images/Screenshot_20240118_004318.png)

Now imagine you want to start another container. You can execute a similar command with just a couple of changes:

```sh
$ podman run -d -p 8081:8080 --name myapp1 \ 
➥ registry.access.redhat.com/ubi8/httpd-24
fa41173e4568a8fa588690d3177150a454c63b53bdfa52865b5f8f7e4d7de1e1
```

Notice you need to change the name of the container to `myapp1`; otherwise, the `podman run command` fails with the `myapp` name because the container previously existed. Also you need to change the `-p` option to use `8081` for the host port because the previous container, myapp, is currently running and is bound to port `8080`. The second container isn’t allowed to bind to port 8080 until the first container exits.

Some notable `podman run` options include the following:

- `--user USERNAME` — This tells Podman to run the container as a specific user defined in the image. By default, Podman will run the container as `root`, unless the container image specifies a default user.
- `--rm` — This automatically removes the container when it exits.
- `--tty` (`-t`) — This allocates a pseudo `-tty` and attaches it to the standard input of the container.
- `--interactive` (`-i`) — This connects stdin to the primary process of the container. These options give you an interactive shell within the container.

> NOTE There are dozens of `podman run` options available, allowing you to change security features, namespaces, volumes, and so on. Some of these I use and explain throughout the book. Refer to the `podman-run` man page for a description of all of the options. Most of the `podman create` options defined in table 2.1 are also available for `podman run`.

Use the `man podman-run` command for information about all options. Now that the container is up and running, it is time to stop the container and go to the next step.

In [1]:
man podman-run

podman-run(1)               General Commands Manual              podman-run(1)

NAME
       podman-run - Run a command in a new container

SYNOPSIS
       podman run [options] image [command [arg ...]]

       podman container run [options] image [command [arg ...]]

DESCRIPTION
       Run  a process in a new container. podman run starts a process with its
       own file system, its own networking, and its own isolated process tree.
       The  image  which starts the process may define defaults related to the
       process that will be run in the container, the  networking  to  expose,
       and  more, but podman run gives final control to the operator or admin‐
       istrator who starts the container from the image. For that reason  pod‐
       man run has more options than any other Podman command.

       If the image is not already loaded then podman run will pull the image,
       and all image dependencies, from the repository in the same way running
       podman pull image

#### `podman create`

The `podman create` command is almost identical to the `podman run` command. The `create` command pulls the image if it is not in container storage and configures the container information to make it ready to run but never executes the container. It is often used together with the `podman start` command described in section 2.1.4. You might want to create a container and then later use a systemd unit file to start and stop the container.

### Stopping containers

#### podman stop

You have two containers running and have tested them by running a web browser against them. To continue the development by actually adding some content to the web page, you can stop the containers using the `podman stop` command:

```sh
$ podman stop myapp
```

The `stop` command stops the container started with the previous `podman run` command.

When stopping a container, Podman examines the running container and sends a stop signal, usually `SIGTERM`, to the primary process (`PID1`) of the container, and then by default it waits `10` seconds for the container to stop. The stop signal tells the primary process within the container to exit gracefully. If the container doesn’t stop within `10` seconds, Podman sends the `SIGKILL` signal to the process, forcing the container to stop. The 10-second wait gives the processes in the container time to clean up and commit changes.

The default stop signal can be changed for a container using the `podman run --stop-signal` option. Sometimes the primary or init process of a container ignores `SIGTERM` (e.g., containers that use systemd as the primary process inside a container). systemd ignores `SIGTERM` and specifies that it shuts down using the `SIGRTMIN+3 `(`signal #37`) signal. The stop signal can be embedded in container images, as I describe in section 2.3.

Some containers ignore the `SIGTERM` stop signal, which means you have to wait 10 seconds for the container to exit. If you know the container ignores the default stop signal, and you don’t care about the container cleaning up, you can just add the `-t 0` option to podman stop to send the `SIGKILL` signal right away:

```sh
$ podman stop -t 0 myapp1
myapp1
```

Some notable `Podman stop` options include the following:

- `--time` (`-t`) — This sets the timeout; `-t 0` sends the `SIGKILL` without waiting for the container to stop.
- `--latest` (`-l`) — This is a useful option to allow you to stop the last created container rather than having to use the container name or container ID. Most Podman commands that require you to specify a container name or ID also accept the `--latest` option. This is only available on Linux machines.
- `--all` — This tells Podman to stop all running containers. Similarly to `--latest`, Podman commands that require a container name or container ID parameter also take the `--all` option.

Use the `man podman-stop` command for information about all options.

In [6]:
man podman-stop

podman-stop(1)              General Commands Manual             podman-stop(1)

NAME
       podman-stop - Stop one or more running containers

SYNOPSIS
       podman stop [options] container ...

       podman container stop [options] container ...

DESCRIPTION
       Stops  one  or  more containers.  You may use container IDs or names as
       input. The --time switch allows you to specify the number of seconds to
       wait  before  forcibly stopping the container after the stop command is
       issued to the container. The default is 10 seconds.  By  default,  con‐
       tainers  are  stopped  with SIGTERM and then SIGKILL after the timeout.
       The SIGTERM default can be overridden by the image used to  create  the
       container and also via command line when creating the container.

OPTIONS
   --all, -a
       Stop all running containers.  This does not include paused containers.

   --cidfile=file
       Read  container ID from the specified file and stop the container.

#### `podman kill`

Podman has a similar command, `podman kill`, which sends the specified kill signal. The `podman kill` command can be useful when you want to send signals into the container without actually stopping the container.

### Starting containers

Eventually, your system will have lots of stopped containers, and sometimes you will need to restart them (e.g., if the system was rebooted). Another common use case is to first `create` a container and later `start` it.

The container you created has now been stopped. Next, you may want to start it back up again using the command in the following listing:

```sh
$ podman start myapp
myapp
```

The `podman start` command starts one or more containers. This command will output the container ID, indicating that your container is up and running. You can now reconnect to it with a web browser. One common use case for `podman start` is starting a container after a reboot to start all of the containers that were stopped during shutdown.

Some favorite Podman start options include these:

- `--all` — This starts all of the stopped containers in container storage.
- `--attach` — This attaches your terminal to the output of the container.
- `--interactive` (`-i`)—This attaches the terminal input to the container.

Use the `man podman-start` command for information about all options.

In [3]:
man podman-start

podman-start(1)             General Commands Manual            podman-start(1)

NAME
       podman-start - Start one or more containers

SYNOPSIS
       podman start [options] container ...

       podman container start [options] container ...

DESCRIPTION
       Start  one  or  more containers.  You may use container IDs or names as
       input.  The attach and interactive options cannot be used  to  override
       the  --tty  and  --interactive options from when the container was cre‐
       ated. If you attempt to start a running container with the --attach op‐
       tion, podman will simply attach to the container.

OPTIONS
   --all
       Start  all  the  containers  created by Podman, default is only running
       containers.

   --attach, -a
       Attach container's STDOUT and STDERR.  The default is false.  This  op‐
       tion cannot be used when starting multiple containers.

   --detach-keys=sequence
       Specify  the key sequence for detaching a container. Format i

### Listing containers

After you’ve been using Podman for a while and have pulled down and run many different container images, you might want to figure out which containers are running or which containers you have in local storage. You will need to be able to list these containers.

Use the `podman ps` command to list containers:

```sh
$ podman ps
```
```
CONTAINER ID IMAGE                      COMMAND        CREATED \ 
➥   STATUS         PORTS          NAMES
b1255e94d084 registry.access.redhat.com/ubi8/httpd-24:latest /usr/bin/run-\ 
➥ http... 6 minutes ago Up 4 minutes ago 0.0.0.0:8080->8080/tcp myapp
```

Notice the `podman ps` command by default lists the running containers. Use the `--all` option to see all of the containers:

```sh
$ podman ps --all
```
```
CONTAINER ID IMAGE                       COMMAND        CREATED \ 
➥   STATUS          PORTS          NAMES
b1255e94d084 registry.access.redhat.com/ubi8/httpd-24:latest /usr/bin/run-\
➥ http... 9 minutes ago Up 8 minutes ago     0.0.0.0:8080->8080/tcp myapp
3efee4d39965 registry.access.redhat.com/ubi8/httpd-24:latest /usr/bin/run-\
➥ http... 7 minutes ago Exited (0) 3 minutes ago 0.0.0.0:8081->8080/tcp myapp1
```

Some notable `podman ps` options include the following:

- `--all` — This tells Podman to list all containers rather than just running containers.
- `--quiet` — This tells Podman to only print the container IDs.
- `--size` — This tells Podman to return the amount of disk space currently used for each container other than the images they are based on.

Use the `man podman-ps` command for information about all options. 

In [7]:
man podman-ps

podman-ps(1)                General Commands Manual               podman-ps(1)

NAME
       podman-ps - Prints out information about containers

SYNOPSIS
       podman ps [options]

       podman container ps [options]

       podman container list [options]

       podman container ls [options]

DESCRIPTION
       podman  ps  lists  the  running containers on the system. Use the --all
       flag to view all the containers information.  By default it lists:

              • container id

              • the name of the image the container is using

              • the COMMAND the container is executing

              • the time the container was created

              • the status of the container

              • port mappings the container is using

              • alternative names for the container

OPTIONS
   --all, -a
       Show all the containers created by Podman, default is only running con‐
       tainers.

       Note: Podman shares containers storage with other tools such